In [ ]:
import numpy as np
np.random.seed(42)

In [2]:
class HiddenMarkovModel:
    def __init__(self, n_states, n_observations):
        self.n_states = n_states
        self.n_observations = n_observations
        
        self.A = np.random.dirichlet(np.ones(n_states), size=n_states)
        
        self.B = np.random.dirichlet(np.ones(n_observations), size=n_states)
        
        self.pi = np.random.dirichlet(np.ones(n_states))
    
    def generate_sequence(self, length):
        states = np.zeros(length, dtype=int)
        observations = np.zeros(length, dtype=int)
        
        states[0] = np.random.choice(self.n_states, p=self.pi)
        observations[0] = np.random.choice(self.n_observations, p=self.B[states[0]])
        
        for t in range(1, length):
            states[t] = np.random.choice(self.n_states, p=self.A[states[t-1]])
            observations[t] = np.random.choice(self.n_observations, p=self.B[states[t]])
            
        return states, observations
    
    def forward_algorithm(self, observations):
        T = len(observations)
        alpha = np.zeros((T, self.n_states))
        
        alpha[0] = self.pi * self.B[:, observations[0]]
        
        # Forward recursion
        for t in range(1, T):
            for j in range(self.n_states):
                alpha[t, j] = self.B[j, observations[t]] * np.sum(alpha[t-1] * self.A[:, j])
                
        return alpha
    
    def viterbi_algorithm(self, observations):
        T = len(observations)
        delta = np.zeros((T, self.n_states))
        psi = np.zeros((T, self.n_states), dtype=int)
        
        delta[0] = self.pi * self.B[:, observations[0]]
        
        for t in range(1, T):
            for j in range(self.n_states):
                temp = delta[t-1] * self.A[:, j]
                delta[t, j] = np.max(temp) * self.B[j, observations[t]]
                psi[t, j] = np.argmax(temp)
        
        states = np.zeros(T, dtype=int)
        states[-1] = np.argmax(delta[-1])
        for t in range(T-2, -1, -1):
            states[t] = psi[t+1, states[t+1]]
            
        return states

In [3]:
n_states = 2
n_observations = 3
sequence_length = 100

hmm = HiddenMarkovModel(n_states, n_observations)

true_states, observations = hmm.generate_sequence(sequence_length)

predicted_states = hmm.viterbi_algorithm(observations)

accuracy = np.mean(true_states == predicted_states)

In [4]:
print(f"Model Parameters:")
print("\nTransition Matrix (A):")
print(hmm.A)
print("\nEmission Matrix (B):")
print(hmm.B)
print("\nInitial State Distribution (π):")
print(hmm.pi)
print(f"\nPrediction Accuracy: {accuracy:.2%}")

print("\nFirst 10 predictions:")
print("True states:      ", true_states[:10])
print("Predicted states: ", predicted_states[:10])

Model Parameters:

Transition Matrix (A):
[[0.13487081 0.86512919]
 [0.59055148 0.40944852]]

Emission Matrix (B):
[[0.42506114 0.42498953 0.14994933]
 [0.48328737 0.22085023 0.29586241]]

Initial State Distribution (π):
[0.00590159 0.99409841]

Prediction Accuracy: 63.00%

First 10 predictions:
True states:       [1 0 1 0 1 0 1 0 1 1]
Predicted states:  [1 0 1 0 1 0 1 0 1 0]
